In [1]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib

In [2]:
with np.load('data/interim/kmers/kmer_matrix.npz') as data:
    kmers = data['kmers']
    kmer_order = data['kmer_order']
    genome_order = data['genome_order']

In [3]:
micsdf = joblib.load('data/interim/mic_class_dataframe2.pkl')
class_orders = joblib.load('data/interim/mic_class_order_dict2.pkl')

In [4]:
# TIO
tio_labels = class_orders['TIO']
tio_label_index = { k: v for v, k in enumerate(tio_labels) }
y_tio = np.array([ tio_label_index[m] if not pd.isna(m) else m for m in micsdf.loc[genome_order, 'TIO'] ])
labels, counts = np.unique(y_tio, return_counts=True)
ok = labels[counts >= 5]

mask = np.in1d(y_tio, ok) # Since Nan is not a label, this also filters invalid MICs
y_tio = y_tio[mask]
X_tio = kmers[mask,:]
tio_samples = genome_order[mask]

In [11]:
# Test/train split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tio, y_tio, test_size=0.2, random_state=36, stratify=y_tio)

joblib.dump(X_train, 'data/interim/X_train_mmap.pkl')
X_train = joblib.load('data/interim/X_train_mmap.pkl', mmap_mode='r+')

In [13]:
# Genetic Algorithm Feature selection
from sklearn import svm
from genetic_selection import GeneticSelectionCV

mod = svm.SVC(C=1.0, kernel='linear')
fs = GeneticSelectionCV(mod,
    cv=5,
    verbose=1,
    scoring="accuracy",
    n_population=50,
    crossover_proba=0.5,
    mutation_proba=0.2,
    n_generations=40,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.05,
    tournament_size=3,
    caching=True,
    n_jobs=1)

fs.fit(X_train, y_train)




Selecting features with genetic algorithm.


In [10]:
fs.generation_scores_

AttributeError: 'GeneticSelectionCV' object has no attribute 'generation_scores_'